In [0]:
# call the heavenly gods
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, bias=False) #input 28x28x1, output 26x26x8, RF 3,
        self.bn1 = nn.BatchNorm2d(8)
        self.d1 = nn.Dropout(0.05) #5 % probability 

        self.conv2 = nn.Conv2d(8, 8, 3, bias=False) #input 26x26x8, output 24x24x8, RF 5
        self.bn2 = nn.BatchNorm2d(8)
        self.d2 = nn.Dropout(0.05) #5 % probability 

        self.conv3 = nn.Conv2d(8, 8, 3, bias=False) #input 24x24x8, output 22x22x8, RF 7
        self.bn3 = nn.BatchNorm2d(8)
        self.d3 = nn.Dropout(0.05) #5 % probability 

        self.pool1 = nn.MaxPool2d(2, 2) # output 11x11x8 RF 8

        self.conv4 = nn.Conv2d(8, 16, 3, bias=False) #input 11x11x8, output 9x9x16, RF 12
        self.bn4 = nn.BatchNorm2d(16)
        self.d4 = nn.Dropout(0.05) #5 % probability 
        
        self.conv5 = nn.Conv2d(16, 16, 3, bias=False) #input 9x9x16, output 7x7x32, RF 16
        self.bn5 = nn.BatchNorm2d(16)
        self.d5 = nn.Dropout(0.05) #5 % probability 

        self.conv6 = nn.Conv2d(16, 16, 3, bias=False) #input 7x7x16, output 5x5x16, RF 20
        self.bn6 = nn.BatchNorm2d(16)
        self.d6 = nn.Dropout(0.05) #5 % probability 

        self.conv7 = nn.Conv2d(16, 32, 3, bias=False) #input 5x5x16, output 3x3x32, RF 24
        self.bn7 = nn.BatchNorm2d(32)
        self.d7 = nn.Dropout(0.05) #5 % probability 
        
        self.conv8 = nn.Conv2d(32, 10, 1, bias=False) #input 3x3x32, output 3x3x10, RF 20

    def forward(self, x):
        x = F.relu(self.bn1(self.d1(self.conv1(x))))
        x = F.relu(self.bn2(self.d2(self.conv2(x))))
        x = self.pool1(F.relu(self.bn3(self.d3(self.conv3(x)))))

        x = F.relu(self.bn4(self.d4(self.conv4(x))))
        x = F.relu(self.bn5(self.d5(self.conv5(x))))
        x = F.relu(self.bn6(self.d6(self.conv6(x))))
        x = self.conv8(F.relu(self.bn7(self.d7(self.conv7(x)))))
        x = F.avg_pool2d(x, 3)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [38]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
           Dropout-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4            [-1, 8, 24, 24]             576
           Dropout-5            [-1, 8, 24, 24]               0
       BatchNorm2d-6            [-1, 8, 24, 24]              16
            Conv2d-7            [-1, 8, 22, 22]             576
           Dropout-8            [-1, 8, 22, 22]               0
       BatchNorm2d-9            [-1, 8, 22, 22]              16
        MaxPool2d-10            [-1, 8, 11, 11]               0
           Conv2d-11             [-1, 16, 9, 9]           1,152
          Dropout-12             [-1, 16, 9, 9]               0
      BatchNorm2d-13             [-1, 16, 9, 9]              32
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    pbar =  tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [41]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in tnrange(1, 21):
    print("\nEpoch:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train set: Average loss: 0.4026, Accuracy: 54043/60000 (90.07%)
Test set: Average loss: 0.0950, Accuracy: 9750/10000 (97.50%)

Epoch: 2


Train set: Average loss: 0.0686, Accuracy: 58881/60000 (98.14%)
Test set: Average loss: 0.0460, Accuracy: 9858/10000 (98.58%)

Epoch: 3


Train set: Average loss: 0.0540, Accuracy: 59064/60000 (98.44%)
Test set: Average loss: 0.0369, Accuracy: 9885/10000 (98.85%)

Epoch: 4


...

Train set: Average loss: 0.0214, Accuracy: 59600/60000 (99.33%)
Test set: Average loss: 0.0225, Accuracy: 9929/10000 (99.29%)

Epoch: 17


Train set: Average loss: 0.0217, Accuracy: 59602/60000 (99.34%)
Test set: Average loss: 0.0201, Accuracy: 9937/10000 (99.37%)

Epoch: 18


Train set: Average loss: 0.0205, Accuracy: 59619/60000 (99.36%)
Test set: Average loss: 0.0206, Accuracy: 9940/10000 (99.40%)

Epoch: 19


Train set: Average loss: 0.0202, Accuracy: 59625/60000 (99.38%)
Test set: Average loss: 0.0204, Accuracy: 9939/10000 (99.39%)

Epoch: 20


Train set: Average loss: 0.0188, Accuracy: 59663/60000 (99.44%)
Test set: Average loss: 0.0208, Accuracy: 9938/10000 (99.38%)
